## Pre-processing :EMA and sensing data
    - slice
    - merge
    - interpolate
    - note: labeling in not included in this process

## process EMA data
    - Original data in json file
    - step 1 to slice json file to csv file by folders, slice by 24 hours, average activities
    - step 2 is to concatenate all csv files into ema_all.csv

In [1]:
#  take a look the raw data structure in EMA
import os
dir_rawdata ='C:/study/studentlife/EMA/response/'  # put the raw data in this place, e.g. c:/study/studentlife
folder_list = os.listdir(dir_rawdata)
print('EMA raw data:', folder_list)
print()
print('E.g., files in /EMA/response/Activity: ', os.listdir(os.path.join(dir_rawdata, 'Activity')))

EMA raw data: ['Activity', 'Administration response', 'Behavior', 'Boston Bombing', 'Cancelled Classes', 'Class', 'Class 2', 'Comment', 'Dartmouth now', 'Dimensions', 'Dimensions protestors', 'Dining Halls', 'Events', 'Exercise', 'Green Key 1', 'Green Key 2', 'Lab', 'Mood', 'Mood 1', 'Mood 2', 'PAM', 'QR_Code', 'Sleep', 'Social', 'Stress', 'Study Spaces']

E.g., files in /EMA/response/Activity:  ['Activity_u00.json', 'Activity_u01.json', 'Activity_u02.json', 'Activity_u03.json', 'Activity_u04.json', 'Activity_u05.json', 'Activity_u07.json', 'Activity_u08.json', 'Activity_u09.json', 'Activity_u10.json', 'Activity_u12.json', 'Activity_u13.json', 'Activity_u14.json', 'Activity_u15.json', 'Activity_u16.json', 'Activity_u17.json', 'Activity_u18.json', 'Activity_u19.json', 'Activity_u20.json', 'Activity_u22.json', 'Activity_u23.json', 'Activity_u24.json', 'Activity_u25.json', 'Activity_u27.json', 'Activity_u30.json', 'Activity_u31.json', 'Activity_u32.json', 'Activity_u33.json', 'Activity_u3

In [6]:
import pandas as pd
import re
import os
from util import stmp_to_fmt, fmt_to_stmp
import numpy as np
import random
dir_data ='../data/'
dir_code ='./'
dir_tosave = '../data/ema'

################ main function of average the acitivities/value by days#################
## concatenate all candicates' acitibity/day by type of activity #########

def count_ema_response(file_dir, folder_name): # count all columns exept ['location','resp_time']
    count=0
    dataset = pd.DataFrame() # creat a empty dataframe
    file_list= sorted(os.listdir(file_dir))
#     file_list=['Activity_u00.json']

    for file in file_list:

        user = re.findall('u[0-9][0-9]', file)[0]
        data = pd.read_json(file_dir+file, convert_dates= False) # keep timestamp formate
        data.fillna(0, inplace =True)
        #print(file,' data sieze: ',data.shape)
        if data.empty:
            continue # if empty go to next file, note; use break will jump out of def
        
        try:
            data= data.drop('location', axis =1) # filter location column
        except: 
            pass
        
        len=data.shape[0]
        if np.any(data.isnull()):
            data=data.dropna(axis=0) # filter null data
        data=data.set_index('resp_time')
        slice_width = 1*3600*24
        for time in range(int(data.index[0]), int(data.index[-1])+1,slice_width):  # timestamp is index, +1 is for data with only one sample
            #data_slice = data.loc[time:time+slice_width-1,:]  # data.loc: slice of both left and right boundary is included, so -1
            slice_index= data.index.where((data.index>= time) & (data.index< time+slice_width)).dropna()
            data_slice =data.loc[slice_index,:]
#             acts=(data_slice.sum(axis =0))  # count the activities in the time slot
            acts=(data_slice.mean(axis =0,skipna=True, numeric_only= True))  # AVERAGE the activities in the time slot
            acts['user']=user
            #acts['timestamp']=time 
#             print(type(user))
            acts['user_date']=user+'_'+stmp_to_fmt(time)[:10] # pickup the yearMMDD
            acts=acts.to_frame().T
 
            dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
            #end of one file processed
        
        count+=1 # end of processing a file
        # end of one folder processed
    
    dataset=dataset.set_index('user_date')
    col=dataset.columns
    new_col=folder_name+'_'+col
    dataset.columns = new_col
    print("Total ",count, " files are processed, in the fold: ", folder_name )
    return dataset

In [3]:
##### A. generate dataset by types for all folders #######
'''
input: raw data in json format under studentlife/ema/response/
output: csv files under ../data/ema/ folder 
'''
import os
import pandas as pd
import re
import os
from util import stmp_to_fmt, fmt_to_stmp
import numpy as np
import random
dir_data ='../data/'
dir_code ='./'
dir_tosave = '../data/ema'

################ main function of average the acitivities/value by days#################
## concatenate all candicates' acitibity/day by type of activity #########

def count_ema_response(file_dir, folder_name): # count all columns exept ['location','resp_time']
    count=0
    dataset = pd.DataFrame() # creat a empty dataframe
    file_list= sorted(os.listdir(file_dir))
#     file_list=['Activity_u00.json']

    for file in file_list:

        user = re.findall('u[0-9][0-9]', file)[0]
        data = pd.read_json(file_dir+file, convert_dates= False) # keep timestamp formate
        data.fillna(0, inplace =True)
        #print(file,' data sieze: ',data.shape)
        if data.empty:
            continue # if empty go to next file, note; use break will jump out of def
        
        try:
            data= data.drop('location', axis =1) # filter location column
        except: 
            pass
        
        len=data.shape[0]
        if np.any(data.isnull()):
            data=data.dropna(axis=0) # filter null data
        data=data.set_index('resp_time')
        slice_width = 1*3600*24
        for time in range(int(data.index[0]), int(data.index[-1])+1,slice_width):  # timestamp is index, +1 is for data with only one sample
            #data_slice = data.loc[time:time+slice_width-1,:]  # data.loc: slice of both left and right boundary is included, so -1
            slice_index= data.index.where((data.index>= time) & (data.index< time+slice_width)).dropna()
            data_slice =data.loc[slice_index,:]
#             acts=(data_slice.sum(axis =0))  # count the activities in the time slot
            acts=(data_slice.mean(axis =0,skipna=True, numeric_only= True))  # AVERAGE the activities in the time slot
            acts['user']=user
            #acts['timestamp']=time 
#             print(type(user))
            acts['user_date']=user+'_'+stmp_to_fmt(time)[:10] # pickup the yearMMDD
            acts=acts.to_frame().T
 
            dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
            #end of one file processed
        
        count+=1 # end of processing a file
        # end of one folder processed
    
    dataset=dataset.set_index('user_date')
    col=dataset.columns
    new_col=folder_name+'_'+col
    dataset.columns = new_col
    print("Total ",count, " files are processed, in the fold: ", folder_name )
    return dataset

dir_data = dir_rawdata
ii=0
folder_list= os.listdir(dir_data)
for folder_name in folder_list: #'activity. behaviour, class ...'
    file_dir = os.path.join(dir_data,folder_name)+'/' # e,g, 'C:/study/studentlife/EMA/response/Activity'
    print(file_dir, folder_name)
    data=count_ema_response(file_dir, folder_name)
    data.to_csv(dir_tosave+folder_name+'.csv')
    ii +=1

print('Total ', ii, ' folders', 'process!')
print("Done! Files saved in  : ", dir_tosave)

C:/study/studentlife/EMA/response/Activity/ Activity
Total  48  files are processed, in the fold:  Activity
C:/study/studentlife/EMA/response/Administration response/ Administration response
Total  26  files are processed, in the fold:  Administration response
C:/study/studentlife/EMA/response/Behavior/ Behavior
Total  47  files are processed, in the fold:  Behavior
C:/study/studentlife/EMA/response/Boston Bombing/ Boston Bombing
Total  11  files are processed, in the fold:  Boston Bombing
C:/study/studentlife/EMA/response/Cancelled Classes/ Cancelled Classes
Total  25  files are processed, in the fold:  Cancelled Classes
C:/study/studentlife/EMA/response/Class/ Class
Total  49  files are processed, in the fold:  Class
C:/study/studentlife/EMA/response/Class 2/ Class 2
Total  33  files are processed, in the fold:  Class 2
C:/study/studentlife/EMA/response/Comment/ Comment
Total  49  files are processed, in the fold:  Comment
C:/study/studentlife/EMA/response/Dartmouth now/ Dartmouth no

In [8]:
########### B. concat all data in file ############
'''
input: csv files under ../data/ema folder 
output:  ema_all.csv file which contains all slices data of ema by users
'''

ema_files_position = dir_tosave
files = os.listdir(ema_files_position)
# files = ['Activity.csv','Class.csv'] # for testing
data_all = pd.DataFrame()
for file in files:
    this_data= pd.read_csv(os.path.join(ema_files_position,file), index_col=0)
    data_all= data_all.merge(this_data, how='outer', left_index=True, right_index=True )
print('Merge success ! Data size of merged dataset is', data_all.shape )
dir_tosave_ema = '../data' # home win10
data_all.to_csv(dir_tosave_ema+ 'ema_all.csv')

Merge success ! Data size of merged dataset is (49, 116)


## process sensing data
    - slice, sum/calculate duration, concatenate
    - process 'activity' and 'audeo' by sum
    - process 'conversation','dark','phonecharge','phonelock' by calculating diration

In [9]:
#  take a look at the raw data structure in Sensing
import os
dir_rawdata ='C:/study/studentlife/sensing/' # put the raw data in this place, e.g. c:/study/studentlife
folder_list = os.listdir(dir_rawdata)
print('Sensing raw data:', folder_list)
print()
print('E.g., files in /Sensing/Audio/: ', os.listdir(os.path.join(dir_rawdata, 'audio')))

Sensing raw data: ['activity', 'audio', 'conversation', 'dark', 'phonecharge', 'phonelock', 'sensing_data_info_summary.csv']

E.g., files in /Sensing/Audio/:  ['audio_u00.csv', 'audio_u01.csv', 'audio_u02.csv', 'audio_u03.csv', 'audio_u04.csv', 'audio_u05.csv', 'audio_u07.csv', 'audio_u08.csv', 'audio_u09.csv', 'audio_u10.csv', 'audio_u12.csv', 'audio_u13.csv', 'audio_u14.csv', 'audio_u15.csv', 'audio_u16.csv', 'audio_u17.csv', 'audio_u18.csv', 'audio_u19.csv', 'audio_u20.csv', 'audio_u22.csv', 'audio_u23.csv', 'audio_u24.csv', 'audio_u25.csv', 'audio_u27.csv', 'audio_u30.csv', 'audio_u31.csv', 'audio_u32.csv', 'audio_u33.csv', 'audio_u34.csv', 'audio_u35.csv', 'audio_u36.csv', 'audio_u39.csv', 'audio_u41.csv', 'audio_u42.csv', 'audio_u43.csv', 'audio_u44.csv', 'audio_u45.csv', 'audio_u46.csv', 'audio_u47.csv', 'audio_u49.csv', 'audio_u50.csv', 'audio_u51.csv', 'audio_u52.csv', 'audio_u53.csv', 'audio_u54.csv', 'audio_u56.csv', 'audio_u57.csv', 'audio_u58.csv', 'audio_u59.csv']


In [14]:
##### 1. take a look at the whole data info in "sensing" folder #######
import os
import pandas as pd
import re
import util
import numpy as np
from util import stmp_to_fmt, fmt_to_stmp
'''
input: raw data in json format under studentlife/sensing/
output: csv files under ../data/sensing/ folder 
'''


dir_rawdata ='C:/study/studentlife/sensing/' # put the raw data in this place, e.g. c:/study/studentlife
dir_code = './'
dir_tosave = '../data/sensing/' 
dir_data = dir_rawdata

def get_file_profile(sub_folder_dir,  file_name):
    dataset = pd.DataFrame() # creat a empty dataframe
    user = re.findall('u[0-9][0-9]', file_name)
    data = pd.read_csv(sub_folder_dir+file_name)
    col=list(data.columns)
    time = data.iloc[:,0]
    data_size = data.shape[0]
    days = int((data.iloc[-1,0]-data.iloc[0,0])/3600/24)

#     data_info_all =pd.DataFrame(columns =['file name','userID','columns','start_time','end_time','duration(days)','instance_no'])
    data_info= [file_name]+user+[','.join(col[:])]+[stmp_to_fmt(int(data.iloc[0,0]))]+[stmp_to_fmt(int(data.iloc[-1,0]))]+[days]+[data_size]
    return data_info

sub_folder_list =[]
folder_list= os.listdir(dir_data)

i=0
data_info_all =pd.DataFrame(columns =['file name','userID','columns','start_time','end_time','duration(days)','instance_no'])
for sub_folder_name in folder_list:  # folder of sensing
    sub_folder_list = sub_folder_list+[sub_folder_name]
    sub_folder_dir = os.path.join(dir_data,sub_folder_name)+'/'
    print(sub_folder_dir)
    file_list = os.listdir(sub_folder_dir)
    for file_name in file_list:
        data_info= get_file_profile(sub_folder_dir,  file_name)
        a_series = pd.Series(data_info, index = data_info_all.columns)
        data_info_all=data_info_all.append(a_series, ignore_index=True)
        i+=1
#         print(file_name)

print('total files: ',i)
print(sub_folder_list)
data_info_all.to_csv(dir_data+"sensing_data_info_summary.csv")
data_info_all

C:/study/studentlife/sensing/activity/
C:/study/studentlife/sensing/audio/
C:/study/studentlife/sensing/conversation/
C:/study/studentlife/sensing/dark/
C:/study/studentlife/sensing/phonecharge/
C:/study/studentlife/sensing/phonelock/
total files:  294
['activity', 'audio', 'conversation', 'dark', 'phonecharge', 'phonelock']


,file name,userID,columns,start_time,end_time,duration(days),instance_no
0,activity_u00.csv,u00,"timestamp, activity inference",2013-03-27 00:00:01,2013-05-31 21:03:33,65,461991
1,activity_u01.csv,u01,"timestamp, activity inference",2013-03-27 00:01:39,2013-05-29 09:32:25,63,474862
2,activity_u02.csv,u02,"timestamp, activity inference",2013-03-27 00:01:48,2013-05-31 23:42:22,65,527518
3,activity_u03.csv,u03,"timestamp, activity inference",2013-03-27 00:02:13,2013-05-22 18:44:33,56,411402
4,activity_u04.csv,u04,"timestamp, activity inference",2013-03-27 00:02:57,2013-05-23 08:28:37,57,462516
...,...,...,...,...,...,...,...
289,phonelock_u54.csv,u54,"start,end",2013-03-27 18:51:36,2013-05-30 05:16:36,63,165
290,phonelock_u56.csv,u56,"start,end",2013-03-27 00:18:53,2013-05-31 12:46:25,65,137
291,phonelock_u57.csv,u57,"start,end",2013-03-27 00:00:05,2013-05-31 16:47:29,65,292
292,phonelock_u58.csv,u58,"start,end",2013-03-27 00:07:50,2013-05-29 07:51:03,63,266


In [11]:
########## 1.2  create a data dictionary for slicing ###########
# start_date = '2013-03-26 00:00:00'
# end_date ='2013-05-31 23:59:59'
# start_date_stmp= fmt_to_stmp(start_date,"%Y-%m-%d %H:%M:%S")
# end_date_stmp= fmt_to_stmp(end_date,"%Y-%m-%d %H:%M:%S")
# # print(start_date_stmp, ' to ',end_date_stmp)

# date_dict={}
# i=0
# for time_stmp in range(start_date_stmp, end_date_stmp, 3600*24):
#     date=stmp_to_fmt(time_stmp)
#     i+=1
#     date_dict[date]=time_stmp, time_stmp+24*3600-1

In [12]:
##########  2.1. process(slice) activity's and audio's: sum activities ######
########### not include wifi , bluetooth, gps. wifi_location#########
'''
input: csv files under ../data/sensing/audio/ folder 
output: audio.csv
'''
def act_count_sliced(dir,file):
    user = re.findall('u[0-9][0-9]', file)[0]
    file= dir+file
    data= pd.read_csv(file, index_col= 0) # index_col= 0: use the first column as the index
#     data.fillna(0, inplace =True)  # if there is null in data set , add this
    if np.any(data.isnull()):
        data=data.dropna(axis=0) # filter null data
    
    slice_width = 1*3600*24
#     time_list=[]
    dataset=pd.DataFrame()
    for time in range(data.index[0], data.index[-1]+1,slice_width):  # timestamp is index
    #     time_list.append(time)
        data_slot = data.loc[time:time+slice_width-1,:]  #note: data.loc: slice of both left and right boundary is included, so -1
        acts=(data_slot.sum(axis =0))  # count the activities in the time slot
        
        #acts['timestamp']=time 
        acts['user']=user
        acts['user_date']=user+'_'+stmp_to_fmt(time)[:10]
        acts=acts.to_frame().T
        dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
        #end of one file processed
        
    dataset=dataset.set_index('user_date')
    dataset=dataset.fillna(0)
#     print(acts)
    return dataset


# dataset =pd.DataFrame()
# dir_temp=(dir_data+'activity/')
# file= 'activity_u00.csv'
# data_s = act_count_sliced(dir_temp,file)
# dataset = pd.concat([dataset,data_s], axis =0) 
# dataset

dataset =pd.DataFrame()
file_dir = dir_data+'activity/'
file_list= sorted(os.listdir(file_dir))
print("total ", len(file_list)," files in the folder of " , file_dir)
print("counting........")
for file in file_list:
    data_= act_count_sliced(file_dir, file)
    dataset = pd.concat([dataset,data_], axis =0) 
col=dataset.columns
new_col=folder+'_'+col
dataset.columns = new_col
print("done!")
dataset.to_csv(dir_tosave+'activity.csv')

dataset =pd.DataFrame()
file_dir = dir_data+'audio/'
file_list= sorted(os.listdir(file_dir))
print("total ", len(file_list)," files in the folder of " , file_dir)
print("counting........")
for file in file_list:
    data_= act_count_sliced(file_dir, file)
    dataset = pd.concat([dataset,data_], axis =0) 
col=dataset.columns
new_col=folder+'_'+col
dataset.columns = new_col
print("done!")
dataset.to_csv(dir_tosave+'audio.csv')

In [13]:
###########  2.2 process (slice) 'conversation','dark','phonecharge','phonelock': calculate duration  ################
'''
input: csv files under ../data/sensing/'conversation','dark','phonecharge','phonelock' folder 
output: csv files of 'conversation','dark','phonecharge','phonelock
'''
def duration_count_sliced(dir,file): #for col 0 is stat_time, col 1 is end_time
    user = re.findall('u[0-9][0-9]', file)[0]
    file= dir+file 
    data= pd.read_csv(file, index_col=0) # index_col= 0: use the first column as the index
    if np.any(data.isnull()):
        data=data.dropna(axis=0) # filter null data
    duration = data.iloc[:,0]-data.index # end_time - start_time
    data.insert(1,'duration(sec)',duration) # add column 
    slice_width = 1*3600*24

#     time_list=[]
    dataset=pd.DataFrame()
    for time in range(int(data.index[0]), int(data.index[-1])+1,slice_width):  # timestamp is index, +1 is for data with only one sample
#         data_slice = data.loc[time:time+slice_width-1,:]  # data.loc: slice of both left and right boundary is included, so -1
        slice_index= data.index.where((data.index>= time) & (data.index< time+slice_width)).dropna()
        data_slice =data.loc[slice_index,:]

        acts=(data_slice.iloc[:,1:].sum(axis =0))  # calculate the total duration in the sliced period, after column 1
        #acts['timestamp']=time 
        acts['user']=user
        acts['user_date']=user+'_'+stmp_to_fmt(time)[:10]        
        acts=acts.to_frame().T
    #     time_list.append(time)
        dataset=pd.concat([dataset, acts], axis =0)  #append the user infor
    dataset=dataset.set_index('user_date')
    dataset=dataset.fillna(0)
    return dataset

# dataset =pd.DataFrame()
# dir_temp=(dir_data+'phonecharge/')
# file= 'phonecharge_u39.csv'

# data_s = duration_count_sliced(dir_temp,file)
# dataset = pd.concat([dataset,data_s], axis =0) 
# dataset

folder_list= ['conversation','dark','phonecharge','phonelock']
# folder_list= ['phonelock']
for folder in folder_list:
    dataset =pd.DataFrame()
    file_dir = dir_data+folder+'/'
    file_list= sorted(os.listdir(file_dir))
    print("total ", len(file_list)," files in the folder of " , file_dir)
    print("counting........")
    for file in file_list:
#         print(file)
        data_= duration_count_sliced(file_dir, file)
        dataset = pd.concat([dataset,data_], axis =0) 
    col=dataset.columns
    new_col=folder+'_'+col
    dataset.columns = new_col
    print("done!")
    dataset.to_csv(dir_tosave+folder+'.csv')
    
##### seperately generate dataset by types for all folders, done ! #######

In [14]:
########### 3. concatenate all sensing data in one file ############
'''
input: csv files of 'activity', audio', 'conversation','dark','phonecharge','phonelock'
output:sensing_all.csv
'''
files_position = dir_tosave #'C:/study/Dropbox/research/depression/data/studentlife/sensing/'
files = os.listdir(files_position) #['Activity.csv','Class.csv', 'Class 2.csv',...]
# files = ['Activity.csv','Class.csv', 'Class 2.csv'] # for testing
data_all = pd.DataFrame()
for file in files:
#     print(file)
    this_data= pd.read_csv(files_position+file, index_col='user_date')
#     print(this_data)
#     data_all=pd.concat([data_all,this_data], axis=1,ignore_index=False)
    data_all= data_all.merge(this_data, how='outer', left_index=True, right_index=True )
print('Merge success ! Data size of merged dataset is', data_all.shape )
data_all
dir_tosave_sensing= '../data/'
data_all.to_csv(dir_tosave_sensing+ 'sensing_all.csv')

## Merge EMA and sensing data

In [4]:
#################  1.1 merge sensing and ema  ####################
######### ema_all_v5.csv, sensing_all_v2.csv  ###################

dir_data = '../data/processed_data/'

dataset_ema = pd.read_csv(dir_data+ 'ema_all_v5.csv', index_col=0)  #v2 is the sliced dataset
print ('1. Dataset_ema shape: ',dataset_ema.shape)

dataset_sensing = pd.read_csv(dir_data+ 'sensing_all_v2.csv', index_col=0)  #v2 is the sliced dataset
print ('2. Dataset_sensing shape: ',dataset_sensing.shape)

######### clean data ###########
dataset= dataset_sensing.merge(dataset_ema, how='outer', left_index=True, right_index=True )
print('3. Merge success ! shape of merged dataset is', dataset.shape )

def drop_notnum(dataset):
    cols=dataset.columns
    col_notnum=[]
    for col in cols:
        if str(dataset[col].dtype)=='object':
               col_notnum.append(col)
    # print(col_notnum)
    dataset=dataset.drop(col_notnum, axis=1)
    return dataset

dataset=drop_notnum(dataset)  #abc=dataset.dropna(axis=1, how='all') # drops columns those with all NaN
dataset_ema= drop_notnum(dataset_ema)
dataset_sensing = drop_notnum(dataset_sensing)
print('')
print('4.1 Removed columns of non-data, merged dataset shape: ', dataset.shape)
print('4.2 Removed columns of non-data, EMA dataset shape: ', dataset_ema.shape)
print('4.2 Removed columns of non-data, sensing dataset shape: ', dataset_sensing.shape)

dataset.to_csv(dir_data+"studentlife_data_full.csv")

1. Dataset_ema shape:  (3025, 107)
2. Dataset_sensing shape:  (3075, 12)
3. Merge success ! shape of merged dataset is (3407, 119)

4.1 Removed columns of non-data, merged dataset shape:  (3407, 87)
4.2 Removed columns of non-data, EMA dataset shape:  (3025, 81)
4.2 Removed columns of non-data, sensing dataset shape:  (3075, 6)
